In [1]:
# Import the libraries you need
import pandas as pd # Pandas is like excel, but in Python
import numpy as np # Numpy is for working with matrices, which is useful for calculating indicator values

In [2]:
# Read in the data
file_path = '../bar_movement/data/' # Goes to the folder where the data is held
currency_pair = 'Eur_Usd' # Currency pair we want to run the simulation for

df = pd.read_csv(file_path + 'Oanda_Eur_Usd_M5_2022-2023.csv') # Reads in the csv file you want
df.Date = pd.to_datetime(df.Date, utc=True) # Make sure the date is an object we can call methods on

In [3]:
# Show the first 5 rows
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume
0,2022-06-15 06:00:00+00:00,1.04625,1.04678,1.04614,1.04637,1.04645,1.04697,1.04635,1.04655,1.04635,1.04688,1.04624,1.04646,1218
1,2022-06-15 06:05:00+00:00,1.04637,1.04651,1.04543,1.04547,1.04654,1.04670,1.04559,1.04564,1.04646,1.04660,1.04552,1.04556,946
2,2022-06-15 06:10:00+00:00,1.04551,1.04594,1.04514,1.04541,1.04569,1.04611,1.04534,1.04558,1.04560,1.04602,1.04524,1.04550,813
3,2022-06-15 06:15:00+00:00,1.04539,1.04630,1.04519,1.04588,1.04558,1.04650,1.04538,1.04607,1.04548,1.04640,1.04529,1.04598,1152
4,2022-06-15 06:20:00+00:00,1.04586,1.04779,1.04582,1.04775,1.04605,1.04797,1.04601,1.04792,1.04596,1.04788,1.04592,1.04784,1271


In [4]:
# Show the last 5 rows
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume
74773,2023-06-15 05:35:00+00:00,1.08162,1.08196,1.08155,1.08165,1.08175,1.08210,1.08171,1.08180,1.08168,1.08203,1.08164,1.08172,353
74774,2023-06-15 05:40:00+00:00,1.08166,1.08188,1.08161,1.08183,1.08182,1.08203,1.08177,1.08196,1.08174,1.08195,1.08169,1.08190,317
74775,2023-06-15 05:45:00+00:00,1.08182,1.08213,1.08180,1.08200,1.08197,1.08228,1.08195,1.08216,1.08190,1.08220,1.08188,1.08208,248
74776,2023-06-15 05:50:00+00:00,1.08200,1.08228,1.08191,1.08198,1.08214,1.08243,1.08206,1.08213,1.08207,1.08236,1.08198,1.08206,333
74777,2023-06-15 05:55:00+00:00,1.08199,1.08228,1.08177,1.08177,1.08213,1.08243,1.08193,1.08193,1.08206,1.08236,1.08185,1.08185,263


In [5]:
# Function for the ATR indicator
def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()

In [6]:
# Add indicators to the dataframe
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

# Make sure there aren't any null values and make sure the row numbers are sequential
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [7]:
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema200,macd,macdsignal,atr
0,2022-06-15 07:05:00+00:00,1.04701,1.04752,1.04631,1.04745,1.04717,1.04768,1.04645,1.04762,1.04709,1.04760,1.04638,1.04754,759,1.047413,0.000057,0.000120,0.001010
1,2022-06-15 07:10:00+00:00,1.04748,1.04812,1.04748,1.04798,1.04763,1.04831,1.04763,1.04815,1.04756,1.04822,1.04756,1.04806,949,1.047459,0.000080,0.000112,0.001013
2,2022-06-15 07:15:00+00:00,1.04798,1.04838,1.04772,1.04798,1.04813,1.04858,1.04790,1.04815,1.04806,1.04848,1.04782,1.04806,953,1.047499,0.000096,0.000109,0.000983
3,2022-06-15 07:20:00+00:00,1.04798,1.04844,1.04758,1.04843,1.04816,1.04862,1.04776,1.04861,1.04807,1.04853,1.04767,1.04852,934,1.047564,0.000136,0.000114,0.000989
4,2022-06-15 07:25:00+00:00,1.04843,1.04873,1.04778,1.04862,1.04863,1.04891,1.04795,1.04877,1.04853,1.04882,1.04786,1.04870,786,1.047633,0.000176,0.000127,0.000978


In [8]:
amounts_per_day = [-0.8, -1, -1.2] if 'Jpy' in currency_pair else [-0.08, -0.1, -0.12]

def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

In [9]:
reward, day_fees, n_buys, n_sells, n_wins, n_losses, longest_win_streak, longest_loss_streak, curr_win_streak, curr_loss_streak = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
trade = None
pips_to_risk = 0.0020 # 20 pips
risk_reward_ratio = 1.5
amount_per_loss = 50
amount_per_win = amount_per_loss * risk_reward_ratio

for i in range(2, len(df)):
    macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
    mid_close, ema200, macd1, macdsignal1 = df.loc[df.index[i - 1], ['Mid_Close', 'ema200', 'macd', 'macdsignal']]
    ask_open, bid_open, ask_high, ask_low, bid_high, bid_low, curr_date = df.loc[df.index[i], ['Ask_Open', 'Bid_Open', 'Ask_High', 'Ask_Low', 'Bid_High', 'Bid_Low', 'Date']]

    # Check if we should open a trade
    if trade is None:
        # For buys
        if mid_close > ema200 and macd2 < macdsignal2 and macd1 > macdsignal1 and max([macd2, macdsignal2, macd1, macdsignal1]) < 0:
            open_price = ask_open
            stop_loss = open_price - pips_to_risk
            take_profit = open_price + (risk_reward_ratio * pips_to_risk)

            trade = {'open_price': open_price, 'stop_loss': stop_loss, 'take_profit': take_profit, 'trade_type': 'buy', 'start_date': curr_date}

            n_buys += 1

        # For sells
        elif mid_close < ema200 and macd2 > macdsignal2 and macd1 < macdsignal1 and min([macd2, macdsignal2, macd1, macdsignal1]) > 0:
            open_price = bid_open
            stop_loss = open_price + pips_to_risk
            take_profit = open_price - (risk_reward_ratio * pips_to_risk)

            trade = {'open_price': open_price, 'stop_loss': stop_loss, 'take_profit': take_profit, 'trade_type': 'sell', 'start_date': curr_date}

            n_sells += 1

    # Check if the trade would've closed out
    if trade is not None:
        if trade['trade_type'] == 'buy' and bid_low < trade['stop_loss']:
            reward -= amount_per_loss # Equivalent to reward = reward - amount_per_loss
            day_fees += calculate_day_fees(trade['start_date'], curr_date)
            n_losses += 1

            curr_win_streak = 0
            curr_loss_streak += 1
            longest_loss_streak = max(longest_loss_streak, curr_loss_streak)

            trade = None # Trade is closed out - move onto the next trade

        elif trade['trade_type'] == 'buy' and bid_high > trade['take_profit']:
            reward += amount_per_win # Equivalent to reward = reward + amount_per_loss
            day_fees += calculate_day_fees(trade['start_date'], curr_date)
            n_wins += 1

            curr_win_streak += 1
            curr_loss_streak = 0
            longest_win_streak = max(longest_win_streak, curr_win_streak)

            trade = None

        elif trade['trade_type'] == 'sell' and ask_high > trade['stop_loss']:
            reward -= amount_per_loss
            day_fees += calculate_day_fees(trade['start_date'], curr_date)
            n_losses += 1

            curr_win_streak = 0
            curr_loss_streak += 1
            longest_loss_streak = max(longest_loss_streak, curr_loss_streak)

            trade = None

        elif trade['trade_type'] == 'sell' and ask_low < trade['take_profit']:
            reward += amount_per_win
            day_fees += calculate_day_fees(trade['start_date'], curr_date)
            n_wins += 1

            curr_win_streak += 1
            curr_loss_streak = 0
            longest_win_streak = max(longest_win_streak, curr_win_streak)

            trade = None

print(f'Reward: {reward}')
print(f'Day fees: {day_fees}')   
print(f'Reward + day fees: {reward + day_fees}')  
print(f'# Buys: {n_buys}')
print(f'# Sells: {n_sells}')
print(f'# Wins: {n_wins}')
print(f'# Losses: {n_losses}')
print(f'Longest win streak: {longest_win_streak}')
print(f'Longest loss streak: {longest_loss_streak}')

Reward: -5125.0
Day fees: -16.59999999999998
Reward + day fees: -5141.6
# Buys: 275
# Sells: 255
# Wins: 171
# Losses: 359
Longest win streak: 5
Longest loss streak: 12
